grasp

In [1]:
import genesis as gs
import numpy as np
import torch
from pytransform3d import (
    transformations as pt,
    rotations as pr,
    batch_rotations as pb,
    trajectories as ptr,
    plot_utils as ppu
)
from pandaSim.geometry.genesis_adapter import GenesisAdapter
import spatialmath as sm
# auto reaload
%load_ext autoreload
%autoreload 2

[I 05/12/25 09:32:22.187 7058] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


In [2]:
gs.init(backend=gs.gpu, seed=42)
adapter = GenesisAdapter()

[Genesis] [09:32:27] [INFO] ╭───────────────────────────────────────────────╮
[Genesis] [09:32:27] [INFO] │┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈ Genesis ┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈│
[Genesis] [09:32:27] [INFO] ╰───────────────────────────────────────────────╯
[Genesis] [09:32:27] [INFO] Running on [NVIDIA GeForce RTX 3070 Laptop GPU] with backend gs.cuda. Device memory: 7.78 GB.
[Genesis] [09:32:27] [INFO] 🚀 Genesis initialized. 🔖 version: 0.2.1, 🌱 seed: 42, 📏 precision: '32', 🐛 debug: False, 🎨 theme: 'dark'.
[Genesis] [09:32:28] [INFO] Scene <cb480ca> created.


In [3]:
scene = adapter.scene
plane = scene.add_entity(
    gs.morphs.Plane(),
)
# robot_path = "../assets/urdf/franka_description/robots/frankaEmikaPandaVirtual.urdf"
# franka_virtual = scene.add_entity(
#     gs.morphs.URDF(file=robot_path, fixed=True),
# )

franka_MJCF = scene.add_entity(
    gs.morphs.MJCF(file="xml/franka_emika_panda/panda.xml",
                   pos = (0.0, 2.0, 0.0)),
    
)
franka = scene.add_entity(
    gs.morphs.MJCF(file="../assets/xml/franka_emika_panda/panda.xml",)
                   
)
cube_size = (0.1, 0.07, 0.03)
cube_pos = [(0.7, 0, 0.015), (0.7, 2.0, 0.015), (0.7, -2.0, 0.015)]
cube, cube2, cube3 = [scene.add_entity(
    gs.morphs.Box(
        size = cube_size,
        pos  = pos,
    ),
    surface=gs.surfaces.Default(
        color=(0.8, 0.1, 0.1),

    ),
    
) for pos in cube_pos]

target_entity = scene.add_entity(
    gs.morphs.Mesh(
        file="meshes/axis.obj",
        scale=0.15,
        collision=False,
    ),
    surface=gs.surfaces.Default(color=(1, 0.5, 0.5, 1)),
)

# cam = scene.add_camera(
#     res    = (640, 480),
#     pos    = (3.5, 0.0, 2.5),
#     lookat = (0, 0, 0.5),
#     fov    = 30,
#     GUI    = False,
# )


[Genesis] [09:32:33] [INFO] Adding <gs.RigidEntity>. idx: 0, uid: <0a18d16>, morph: <gs.morphs.Plane>, material: <gs.materials.Rigid>.
[Genesis] [09:32:34] [INFO] Adding <gs.RigidEntity>. idx: 1, uid: <19b3233>, morph: <gs.morphs.MJCF(file='/home/vahid/envs/panda/lib/python3.10/site-packages/genesis/assets/xml/franka_emika_panda/panda.xml')>, material: <gs.materials.Rigid>.
[Genesis] [09:32:34] [WARNING] (MJCF) Tendon not supported
[Genesis] [09:32:34] [WARNING] (MJCF) Approximating tendon by joint actuator for `finger_joint1`
[Genesis] [09:32:34] [WARNING] (MJCF) Actuator control gain and bias parameters cannot be reduced to a unique PD control position gain. Using max between gain and bias.
[Genesis] [09:32:34] [WARNING] (MJCF) Approximating tendon by joint actuator for `finger_joint2`
[Genesis] [09:32:34] [WARNING] (MJCF) Actuator control gain and bias parameters cannot be reduced to a unique PD control position gain. Using max between gain and bias.
[Genesis] [09:32:34] [INFO] Appl

In [4]:
n_envs = 0
scene.build()
motors_dof = np.arange(7)
fingers_dof = np.arange(7, 9)

[Genesis] [09:32:42] [INFO] Building scene <cb480ca>...
[Genesis] [09:32:44] [WARNING] Reference robot position exceeds joint limits.
[Genesis] [09:32:44] [WARNING] Constraint solver time constant was increased to avoid numerical instability (from `0.005` to `0.02`). Decrease simulation timestep to avoid altering the original value.


[W 05/12/25 09:32:45.779 7058] [frontend_ir.cpp:begin_frontend_struct_for_on_external_tensor@1694] ti.loop_config(serialize=True) does not have effect on the struct for. The execution order is not guaranteed.


[Genesis] [09:32:52] [INFO] Compiling simulation kernels...
[Genesis] [09:36:00] [INFO] Building visualizer...
[Genesis] [09:36:04] [INFO] Viewer created. Resolution: 1280×960, max_FPS: 60.


In [5]:
franka.n_dofs, franka_MJCF.n_dofs


(9, 9)

In [30]:
scene.reset()

[Genesis] [01:43:09] [INFO] Resetting Scene <0b45cdc>.


In [35]:
qr = np.array([0.0000, -0.3002, 0.0000, -2.1991, 0.0000, 2.0071, 0.7854, 0.04, 0.04])

franka.ready_qpos = qr
franka.set_qpos(franka.ready_qpos)
franka.control_dofs_position(franka.ready_qpos)
franka_MJCF.set_qpos(franka.ready_qpos)
franka_MJCF.control_dofs_position(franka.ready_qpos)

end_effector = franka.get_link('hand')

scene.step()

[Genesis] [10:26:46] [INFO] Running at 0.01 FPS.


In [32]:
for _ in range(100):
    scene.step()

[Genesis] [01:43:13] [INFO] Running at 0.27 FPS.
[Genesis] [01:43:13] [INFO] Running at 0.29 FPS.
[Genesis] [01:43:13] [INFO] Running at 0.30 FPS.
[Genesis] [01:43:13] [INFO] Running at 0.32 FPS.
[Genesis] [01:43:13] [INFO] Running at 0.33 FPS.
[Genesis] [01:43:13] [INFO] Running at 0.35 FPS.
[Genesis] [01:43:13] [INFO] Running at 0.37 FPS.
[Genesis] [01:43:13] [INFO] Running at 0.39 FPS.
[Genesis] [01:43:13] [INFO] Running at 0.41 FPS.
[Genesis] [01:43:13] [INFO] Running at 0.43 FPS.
[Genesis] [01:43:13] [INFO] Running at 0.45 FPS.
[Genesis] [01:43:13] [INFO] Running at 0.48 FPS.
[Genesis] [01:43:13] [INFO] Running at 0.50 FPS.
[Genesis] [01:43:13] [INFO] Running at 0.53 FPS.
[Genesis] [01:43:13] [INFO] Running at 0.56 FPS.
[Genesis] [01:43:13] [INFO] Running at 0.58 FPS.
[Genesis] [01:43:13] [INFO] Running at 0.61 FPS.
[Genesis] [01:43:13] [INFO] Running at 0.65 FPS.
[Genesis] [01:43:13] [INFO] Running at 0.68 FPS.
[Genesis] [01:43:13] [INFO] Running at 0.72 FPS.
[Genesis] [01:43:13]

In [33]:
target_entity.set_pos(end_effector.get_pos())
target_entity.set_quat(end_effector.get_quat())
scene.step()

[Genesis] [01:43:27] [INFO] Running at 1.54 FPS.


In [ ]:
# target_quat = np.array([0, 1, 0, 0])
# center = np.array([0.5, 0, 0.5])
# r = 0.1
# damping = 1e-6
# diag = damping * np.eye(6)
# target_pos = center + np.array([1, 0, 0]) * r
# init_q = franka.inverse_kinematics(
#     link = end_effector,
#     pos  = target_pos,
#     quat = target_quat,
# )
# franka.set_qpos(init_q)
# franka.control_dofs_position(init_q)
# for i in range(100):
#     target_entity.set_qpos(np.concatenate([target_pos, target_quat]))
#     scene.step()

# for i in range(0, 500):
#     target_pos = center + np.array([np.cos(i / 360 * np.pi), np.sin(i / 360 * np.pi), 0]) * r

#     target_entity.set_qpos(np.concatenate([target_pos, target_quat]))

#     # Position error.
#     error_pos = target_pos - end_effector.get_pos().cpu().numpy()

#     # Orientation error.
#     ee_quat = end_effector.get_quat().cpu().numpy()
#     error_quat = gs.transform_quat_by_quat(gs.inv_quat(ee_quat), target_quat)
#     error_rotvec = gs.quat_to_rotvec(error_quat)

#     error = np.concatenate([error_pos, error_rotvec])

#     # jacobian
#     jac = franka.get_jacobian(link=end_effector).cpu().numpy()
#     dq = jac.T @ np.linalg.solve(jac @ jac.T + diag, error)
#     q = franka.get_qpos().cpu().numpy() + dq

#     # control
#     franka.control_dofs_position(q)
#     scene.step()


In [34]:

# get the end-effector link
end_effector = franka.get_link('hand')
end_effector_MJCF = franka_MJCF.get_link('hand')
above_box = cube.get_pos().cpu().numpy() + np.array([-cube_size[0]/2 + 0.02, 0, 0.11 + 0.1])
# move to pre-grasp pose
qpos = franka.inverse_kinematics(
    link = end_effector,
    pos  = above_box,
    quat = np.array([0, 1, 0, 0]),
)
qpos_MJCF = franka_MJCF.inverse_kinematics(
    link = end_effector_MJCF,
    pos  = above_box + np.array([0.0, 2.0, 0.0]),
    quat = np.array([0, 1, 0, 0]),
)
# gripper open pos
qpos[-2:] = 0.04
qpos_MJCF[-2:] = 0.04
path = franka.plan_path(
    qpos_goal     = qpos,
    num_waypoints = 200, # 2s duration
)
path_MJCF = franka_MJCF.plan_path(
    qpos_goal     = qpos_MJCF,
    num_waypoints = 200, # 2s duration
)

[Genesis] [01:43:38] [INFO] Path solution found successfully.
[Genesis] [01:43:38] [INFO] Path solution found successfully.


In [35]:
# execute the planned path
for waypoint, waypoint_MJCF in zip(path, path_MJCF):
    franka.control_dofs_position(waypoint)
    franka_MJCF.control_dofs_position(waypoint_MJCF)
    target_entity.set_pos(end_effector.get_pos())
    target_entity.set_quat(end_effector.get_quat())
    scene.step()

# allow robot to reach the last waypoint
for i in range(100):
    target_entity.set_pos(end_effector.get_pos())
    target_entity.set_quat(end_effector.get_quat())
    scene.step()


[Genesis] [01:43:41] [INFO] Running at 0.77 FPS.
[Genesis] [01:43:41] [INFO] Running at 0.81 FPS.
[Genesis] [01:43:41] [INFO] Running at 0.85 FPS.
[Genesis] [01:43:41] [INFO] Running at 0.89 FPS.
[Genesis] [01:43:41] [INFO] Running at 0.94 FPS.
[Genesis] [01:43:41] [INFO] Running at 0.98 FPS.
[Genesis] [01:43:41] [INFO] Running at 1.03 FPS.
[Genesis] [01:43:41] [INFO] Running at 1.09 FPS.
[Genesis] [01:43:41] [INFO] Running at 1.14 FPS.
[Genesis] [01:43:41] [INFO] Running at 1.20 FPS.
[Genesis] [01:43:41] [INFO] Running at 1.26 FPS.
[Genesis] [01:43:42] [INFO] Running at 1.32 FPS.
[Genesis] [01:43:42] [INFO] Running at 1.39 FPS.
[Genesis] [01:43:42] [INFO] Running at 1.46 FPS.
[Genesis] [01:43:42] [INFO] Running at 1.53 FPS.
[Genesis] [01:43:42] [INFO] Running at 1.61 FPS.
[Genesis] [01:43:42] [INFO] Running at 1.69 FPS.
[Genesis] [01:43:42] [INFO] Running at 1.77 FPS.
[Genesis] [01:43:42] [INFO] Running at 1.86 FPS.
[Genesis] [01:43:42] [INFO] Running at 1.95 FPS.
[Genesis] [01:43:42]

In [36]:
side_box = above_box + np.array([0.0, 0.0, -0.1])


# reach
qpos_reach = franka.inverse_kinematics(
    link = end_effector,
    pos  = side_box,
    quat = np.array([0, 1, 0, 0]),
)
qpos_reach_MJCF = franka_MJCF.inverse_kinematics(
    link = end_effector_MJCF,
    pos  = side_box + np.array([0.0, 2.0, 0.0]),
    quat = np.array([0, 1, 0, 0]),
)
path = franka.plan_path(
    qpos_goal     = qpos_reach,
    num_waypoints = 100, # 1s duration
)
path_MJCF = franka_MJCF.plan_path(
    qpos_goal     = qpos_reach_MJCF,
    num_waypoints = 100, # 1s duration
)

for waypoint, waypoint_MJCF in zip(path, path_MJCF):
    franka.control_dofs_position(waypoint)
    franka_MJCF.control_dofs_position(waypoint_MJCF)
    target_entity.set_pos(end_effector.get_pos())
    target_entity.set_quat(end_effector.get_quat())
    scene.step()

# allow robot to reach the last waypoint
for i in range(100):
    target_entity.set_pos(end_effector.get_pos())
    target_entity.set_quat(end_effector.get_quat())
    scene.step()

[Genesis] [01:43:59] [INFO] Path solution found successfully.
[Genesis] [01:44:00] [INFO] Path solution found successfully.
[Genesis] [01:44:00] [INFO] Running at 2.15 FPS.
[Genesis] [01:44:00] [INFO] Running at 2.26 FPS.
[Genesis] [01:44:00] [INFO] Running at 2.37 FPS.
[Genesis] [01:44:00] [INFO] Running at 2.48 FPS.
[Genesis] [01:44:00] [INFO] Running at 2.60 FPS.
[Genesis] [01:44:00] [INFO] Running at 2.73 FPS.
[Genesis] [01:44:00] [INFO] Running at 2.86 FPS.
[Genesis] [01:44:00] [INFO] Running at 2.99 FPS.
[Genesis] [01:44:00] [INFO] Running at 3.13 FPS.
[Genesis] [01:44:00] [INFO] Running at 3.28 FPS.
[Genesis] [01:44:00] [INFO] Running at 3.43 FPS.
[Genesis] [01:44:00] [INFO] Running at 3.58 FPS.
[Genesis] [01:44:00] [INFO] Running at 3.75 FPS.
[Genesis] [01:44:00] [INFO] Running at 3.92 FPS.
[Genesis] [01:44:00] [INFO] Running at 4.10 FPS.
[Genesis] [01:44:00] [INFO] Running at 4.29 FPS.
[Genesis] [01:44:00] [INFO] Running at 4.48 FPS.
[Genesis] [01:44:00] [INFO] Running at 4.68

In [37]:



# grasp
franka.control_dofs_position(qpos_reach[:-2], motors_dof)
franka.control_dofs_force(np.array([-5, -5]), fingers_dof)
franka_MJCF.control_dofs_position(qpos_reach_MJCF[:-2], motors_dof)
franka_MJCF.control_dofs_force(np.array([-5, -5]), fingers_dof)

for i in range(100):
    target_entity.set_pos(end_effector.get_pos())
    target_entity.set_quat(end_effector.get_quat())
    scene.step()


[Genesis] [01:44:16] [INFO] Running at 1.98 FPS.
[Genesis] [01:44:16] [INFO] Running at 2.08 FPS.
[Genesis] [01:44:16] [INFO] Running at 2.16 FPS.
[Genesis] [01:44:16] [INFO] Running at 2.23 FPS.
[Genesis] [01:44:16] [INFO] Running at 2.32 FPS.
[Genesis] [01:44:16] [INFO] Running at 2.40 FPS.
[Genesis] [01:44:16] [INFO] Running at 2.49 FPS.
[Genesis] [01:44:17] [INFO] Running at 2.58 FPS.
[Genesis] [01:44:17] [INFO] Running at 2.67 FPS.
[Genesis] [01:44:17] [INFO] Running at 2.77 FPS.
[Genesis] [01:44:17] [INFO] Running at 2.86 FPS.
[Genesis] [01:44:17] [INFO] Running at 2.96 FPS.
[Genesis] [01:44:17] [INFO] Running at 3.06 FPS.
[Genesis] [01:44:17] [INFO] Running at 3.16 FPS.
[Genesis] [01:44:17] [INFO] Running at 3.26 FPS.
[Genesis] [01:44:17] [INFO] Running at 3.37 FPS.
[Genesis] [01:44:18] [INFO] Running at 3.48 FPS.
[Genesis] [01:44:18] [INFO] Running at 3.60 FPS.
[Genesis] [01:44:18] [INFO] Running at 3.71 FPS.
[Genesis] [01:44:18] [INFO] Running at 3.83 FPS.
[Genesis] [01:44:18]

In [38]:

# lift
# move to pre-grasp pose
qpos = franka.inverse_kinematics(
    link = end_effector,
    pos  = above_box,
    quat = np.array([0, 1, 0, 0]),
)
qpos_MJCF = franka_MJCF.inverse_kinematics(
    link = end_effector_MJCF,
    pos  = above_box + np.array([0.0, 2.0, 0.0]),
    quat = np.array([0, 1, 0, 0]),
)
franka.control_dofs_position(qpos[:-2], motors_dof)
franka_MJCF.control_dofs_position(qpos_MJCF[:-2], motors_dof)

for i in range(200):
    target_entity.set_pos(end_effector.get_pos())
    target_entity.set_quat(end_effector.get_quat())
    scene.step()

[Genesis] [01:44:38] [INFO] Running at 1.56 FPS.
[Genesis] [01:44:38] [INFO] Running at 1.63 FPS.
[Genesis] [01:44:38] [INFO] Running at 1.70 FPS.
[Genesis] [01:44:38] [INFO] Running at 1.77 FPS.
[Genesis] [01:44:38] [INFO] Running at 1.85 FPS.
[Genesis] [01:44:38] [INFO] Running at 1.93 FPS.
[Genesis] [01:44:38] [INFO] Running at 2.01 FPS.
[Genesis] [01:44:38] [INFO] Running at 2.09 FPS.
[Genesis] [01:44:39] [INFO] Running at 2.18 FPS.
[Genesis] [01:44:39] [INFO] Running at 2.26 FPS.
[Genesis] [01:44:39] [INFO] Running at 2.36 FPS.
[Genesis] [01:44:39] [INFO] Running at 2.45 FPS.
[Genesis] [01:44:39] [INFO] Running at 2.55 FPS.
[Genesis] [01:44:39] [INFO] Running at 2.64 FPS.
[Genesis] [01:44:39] [INFO] Running at 2.74 FPS.
[Genesis] [01:44:39] [INFO] Running at 2.85 FPS.
[Genesis] [01:44:39] [INFO] Running at 2.95 FPS.
[Genesis] [01:44:39] [INFO] Running at 3.06 FPS.
[Genesis] [01:44:40] [INFO] Running at 3.17 FPS.
[Genesis] [01:44:40] [INFO] Running at 3.28 FPS.
[Genesis] [01:44:40]

In [40]:
ff = 0
franka.control_dofs_force(np.array([ff, ff]), fingers_dof)
franka_MJCF.control_dofs_force(np.array([ff, ff]), fingers_dof)
for _ in range(500):
    scene.step()

[Genesis] [01:50:38] [INFO] Running at 0.10 FPS.
[Genesis] [01:50:38] [INFO] Running at 0.10 FPS.
[Genesis] [01:50:38] [INFO] Running at 0.11 FPS.
[Genesis] [01:50:38] [INFO] Running at 0.11 FPS.
[Genesis] [01:50:38] [INFO] Running at 0.12 FPS.
[Genesis] [01:50:38] [INFO] Running at 0.13 FPS.
[Genesis] [01:50:38] [INFO] Running at 0.13 FPS.
[Genesis] [01:50:38] [INFO] Running at 0.14 FPS.
[Genesis] [01:50:38] [INFO] Running at 0.15 FPS.
[Genesis] [01:50:38] [INFO] Running at 0.15 FPS.
[Genesis] [01:50:38] [INFO] Running at 0.16 FPS.
[Genesis] [01:50:38] [INFO] Running at 0.17 FPS.
[Genesis] [01:50:38] [INFO] Running at 0.18 FPS.
[Genesis] [01:50:39] [INFO] Running at 0.19 FPS.
[Genesis] [01:50:39] [INFO] Running at 0.20 FPS.
[Genesis] [01:50:39] [INFO] Running at 0.21 FPS.
[Genesis] [01:50:39] [INFO] Running at 0.22 FPS.
[Genesis] [01:50:39] [INFO] Running at 0.23 FPS.
[Genesis] [01:50:39] [INFO] Running at 0.25 FPS.
[Genesis] [01:50:39] [INFO] Running at 0.26 FPS.
[Genesis] [01:50:39]